In [ ]:
import requests
import csv
import time

API_KEY = ""
STEAM_ID = ""

#  보유 게임 가져오기
owned_url = "https://api.steampowered.com/IPlayerService/GetOwnedGames/v1/"
params = {
    "key": API_KEY,
    "steamid": STEAM_ID,
    "include_appinfo": True,
    "include_played_free_games": True
}

res = requests.get(owned_url, params=params).json()
games = res["response"]["games"]

#  플레이타임 기준 정렬 (내림차순)
games.sort(key=lambda g: g["playtime_forever"], reverse=True)

#  CSV 저장
with open("steam_games_full.csv", "w", newline="", encoding="utf-8-sig") as f:
    writer = csv.writer(f)
    writer.writerow([
        "appid",
        "game_name",
        "playtime_total_hours",
        "playtime_2weeks_hours",
        "developer",
        "price_krw",
        "store_url"
    ])

    for g in games:
        appid = g["appid"]
        name = g["name"]

        # Store API (개발사 / 가격)
        store_url = f"https://store.steampowered.com/app/{appid}"
        store_api = f"https://store.steampowered.com/api/appdetails?appids={appid}&cc=kr&l=ko"

        developer = ""
        price = "FREE"

        try:
            store_res = requests.get(store_api, timeout=5).json()
            data = store_res.get(str(appid), {}).get("data", {})

            if data:
                developer = ", ".join(data.get("developers", []))

                if data.get("is_free"):
                    price = "FREE"
                elif "price_overview" in data:
                    price = data["price_overview"]["final"] / 100
        except:
            pass

        writer.writerow([
            appid,
            name,
            round(g["playtime_forever"] / 60, 2),
            round(g.get("playtime_2weeks", 0) / 60, 2),
            developer,
            price,
            store_url
        ])

        # 속도 제한 (Steam 차단 방지)
        time.sleep(0.3)

print("steam_games_Timelist.csv 저장 완료")

In [ ]:
import os
os.getcwd()


In [ ]:
import requests
import csv
import time

API_KEY = "key"
STEAM_ID = "id"

#  보유 게임 가져오기
owned_url = "https://api.steampowered.com/IPlayerService/GetOwnedGames/v1/"
params = {
    "key": API_KEY,
    "steamid": STEAM_ID,
    "include_appinfo": True,
    "include_played_free_games": True
}

res = requests.get(owned_url, params=params).json()
games = res["response"]["games"]

#  총 플레이타임 기준 정렬
games.sort(key=lambda g: g["playtime_forever"], reverse=True)

with open("steam_games_full_v1.csv", "w", newline="", encoding="utf-8-sig") as f:
    writer = csv.writer(f)
    writer.writerow([
        "appid",
        "game_name",
        "playtime_total_hours",
        "playtime_2weeks_hours",
        "recent_played",
        "price_krw",
        "hours_per_price",
        "is_free",
        "developer",
        "publisher",
        "genres",
        "release_date",
        "korean_support",
        "metacritic_score",
        "store_url"
    ])

    for g in games:
        appid = g["appid"]
        name = g["name"]

        total_hours = round(g["playtime_forever"] / 60, 2)
        recent_hours = round(g.get("playtime_2weeks", 0) / 60, 2)
        recent_played = "YES" if recent_hours > 0 else "NO"

        # 기본값
        price = ""
        is_free = False
        developer = ""
        publisher = ""
        genres = ""
        release_date = ""
        korean_support = "NO"
        metacritic = ""

        store_url = f"https://store.steampowered.com/app/{appid}"
        store_api = f"https://store.steampowered.com/api/appdetails?appids={appid}&cc=kr&l=ko"

        try:
            store_res = requests.get(store_api, timeout=5).json()
            data = store_res.get(str(appid), {}).get("data", {})

            if data:
                # 개발사 / 퍼블리셔
                developer = ", ".join(data.get("developers", []))
                publisher = ", ".join(data.get("publishers", []))

                # 무료 여부 + 가격
                is_free = data.get("is_free", False)
                if is_free:
                    price = 0
                elif "price_overview" in data:
                    price = data["price_overview"]["final"] / 100

                # 장르
                if "genres" in data:
                    genres = ", ".join(g["description"] for g in data["genres"])

                # 출시일
                release_date = data.get("release_date", {}).get("date", "")

                # 한국어 지원
                if "Korean" in data.get("supported_languages", ""):
                    korean_support = "YES"

                # 메타크리틱 점수
                metacritic = data.get("metacritic", {}).get("score", "")

        except:
            pass

        # 플레이 효율 계산
        if price and price > 0:
            hours_per_price = round(total_hours / price, 3)
        else:
            hours_per_price = ""

        writer.writerow([
            appid,
            name,
            total_hours,
            recent_hours,
            recent_played,
            price,
            hours_per_price,
            is_free,
            developer,
            publisher,
            genres,
            release_date,
            korean_support,
            metacritic,
            store_url
        ])

        #  API 과부하 방지
        time.sleep(0.3)

print("steam_games_v1.csv 생성 완료")


steam_games_v1.csv 생성 완료


In [ ]:
pip install openpyxl


     -------------------------------------- 250.9/250.9 kB 7.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import requests
import time
from openpyxl import Workbook
from openpyxl.styles import Alignment
from openpyxl.utils import get_column_letter

API_KEY = "KEY"
STEAM_ID = "Name"

# 보유 게임 가져오기
owned_url = "https://api.steampowered.com/IPlayerService/GetOwnedGames/v1/"
params = {
    "key": API_KEY,
    "steamid": STEAM_ID,
    "include_appinfo": True,
    "include_played_free_games": True
}

res = requests.get(owned_url, params=params).json()
games = res["response"]["games"]

# 총 플레이타임 기준 정렬
games.sort(key=lambda g: g["playtime_forever"], reverse=True)

# 엑셀 워크북 생성
wb = Workbook()
ws = wb.active
ws.title = "Steam Games"

headers = [
    "appid",
    "game_name",
    "playtime_total_hours",
    "playtime_2weeks_hours",
    "recent_played",
    "price_krw",
    "hours_per_price",
    "is_free",
    "developer",
    "publisher",
    "genres",
    "release_date",
    "korean_support",
    "metacritic_score",
    "store_url"
]

ws.append(headers)

center_align = Alignment(horizontal="center", vertical="center")

# 헤더 가운데 정렬
for col in range(1, len(headers) + 1):
    ws.cell(row=1, column=col).alignment = center_align

# 데이터 수집
for g in games:
    appid = g["appid"]
    name = g["name"]

    total_hours = round(g["playtime_forever"] / 60, 2)
    recent_hours = round(g.get("playtime_2weeks", 0) / 60, 2)
    recent_played = "YES" if recent_hours > 0 else "NO"

    price = ""
    is_free = False
    developer = ""
    publisher = ""
    genres = ""
    release_date = ""
    korean_support = "NO"
    metacritic = ""

    store_url = f"https://store.steampowered.com/app/{appid}"
    store_api = f"https://store.steampowered.com/api/appdetails?appids={appid}&cc=kr&l=ko"

    try:
        store_res = requests.get(store_api, timeout=5).json()
        data = store_res.get(str(appid), {}).get("data", {})

        if data:
            developer = ", ".join(data.get("developers", []))
            publisher = ", ".join(data.get("publishers", []))

            is_free = data.get("is_free", False)
            if is_free:
                price = 0
            elif "price_overview" in data:
                price = data["price_overview"]["final"] / 100

            if "genres" in data:
                genres = ", ".join(x["description"] for x in data["genres"])

            release_date = data.get("release_date", {}).get("date", "")

            if "Korean" in data.get("supported_languages", ""):
                korean_support = "YES"

            metacritic = data.get("metacritic", {}).get("score", "")

    except:
        pass

    if price and price > 0:
        hours_per_price = round(total_hours / price, 3)
    else:
        hours_per_price = ""

    row = [
        appid,
        name,
        total_hours,
        recent_hours,
        recent_played,
        price,
        hours_per_price,
        is_free,
        developer,
        publisher,
        genres,
        release_date,
        korean_support,
        metacritic,
        store_url
    ]

    ws.append(row)

    # 가운데 정렬
    row_idx = ws.max_row
    for col in range(1, len(headers) + 1):
        ws.cell(row=row_idx, column=col).alignment = center_align

    time.sleep(0.3)

#  열 너비 자동 조절
for col in ws.columns:
    max_length = 0
    col_letter = get_column_letter(col[0].column)
    for cell in col:
        if cell.value:
            max_length = max(max_length, len(str(cell.value)))
    ws.column_dimensions[col_letter].width = max_length + 2

#  저장
wb.save("steam_games_full_v1.xlsx")
print("steam_games_full_v1.xlsx 생성 완료")


steam_games_full_v1.xlsx 생성 완료


In [ ]:
# steam_to_xlsx_full.py
# 필요: pip install requests openpyxl
import requests
import time
from openpyxl import Workbook
from openpyxl.styles import Alignment
from openpyxl.utils import get_column_letter

# --------- 설정 (정보 기입 / git 업로드 시 정보 제거 ) ----------
API_KEY = "YOUR_API_KEY"  # "YOUR_API_KEY"에 스팀 api키 입력
STEAM_ID = "YOUR_STEAM_ID"               # "YOUR_STEAM_ID"에 유저 번호 입력
OUTPUT_XLSX = "Name.xlsx" # 출력할 파일 이름

# 제외할 AppID들을 넣으면 해당 항목은 CSV/XLSX에 기록되지 않음
EXCLUDE_APPIDS = {
    # 예: 228980, 1070560
}

# 요청에 사용할 공통 헤더 (Store API / Web API에서 봇 차단 방지)
REQ_HEADERS = {"User-Agent": "Mozilla/5.0 (compatible; SteamDataScript/1.0)"}
# ----------------------------------------

def safe_get_json(url, params=None, headers=None, timeout=8):
    try:
        r = requests.get(url, params=params, headers=headers or REQ_HEADERS, timeout=timeout)
        r.raise_for_status()
        # 방어적: 빈 응답 또는 HTML이면 빈 dict 반환
        txt = r.text.strip()
        if not txt:
            return {}
        # JSON 파싱 시도
        try:
            return r.json()
        except ValueError:
            return {}
    except Exception as e:
        print("Request error:", e, "url:", url, "params:", params)
        return {}

def main():
    print("1) GetOwnedGames 호출 중...")
    owned_url = "https://api.steampowered.com/IPlayerService/GetOwnedGames/v1/"
    params = {
        "key": API_KEY,
        "steamid": STEAM_ID,
        "include_appinfo": True,
        "include_played_free_games": True
    }

    owned_res = safe_get_json(owned_url, params=params)
    resp = owned_res.get("response", {})
    games = resp.get("games", [])

    if not games:
        print("소유 게임 목록을 불러오지 못했거나 비어 있습니다. (프로필 공개 여부 / API_KEY 확인)")
        return

    # 플레이타임 기준 내림차순 정렬
    games.sort(key=lambda g: g.get("playtime_forever", 0), reverse=True)
    print(f"소유 게임 수: {len(games)} (정렬 완료)")

    # 엑셀 생성
    wb = Workbook()
    ws = wb.active
    ws.title = "Steam Games"

    headers = [
        "appid",
        "game_name",
        "playtime_total_hours",
        "playtime_2weeks_hours",
        "recent_played",
        "price_krw",
        "hours_per_price",
        "is_free",
        "sale_status",       # PAID / FREE / DELISTED / UNKNOWN
        "developer",
        "publisher",
        "genres",
        "release_date",
        "korean_support",
        "metacritic_score",
        "store_url"
    ]
    ws.append(headers)

    center_align = Alignment(horizontal="center", vertical="center")

    total = len(games)
    for idx, g in enumerate(games, start=1):
        appid = g.get("appid")
        # AppID 기준 제외
        if appid in EXCLUDE_APPIDS:
            print(f"[{idx}/{total}] 제외된 AppID {appid} 스킵")
            continue

        name = g.get("name", "")
        total_hours = round(g.get("playtime_forever", 0) / 60, 2)
        recent_hours = round(g.get("playtime_2weeks", 0) / 60, 2)
        recent_played = "YES" if recent_hours > 0 else "NO"

        # 기본값
        price = ""
        is_free = False
        sale_status = "UNKNOWN"
        developer = ""
        publisher = ""
        genres = ""
        release_date = ""
        korean_support = "NO"
        metacritic = ""

        store_url = f"https://store.steampowered.com/app/{appid}"
        store_api = f"https://store.steampowered.com/api/appdetails"
        # store API는 appids 파라미터에 숫자 넣음
        params_store = {"appids": appid, "cc": "kr", "l": "ko"}

        # Store 조회
        store_json = safe_get_json(store_api, params=params_store)
        data_wrapper = store_json.get(str(appid), {})
        data = data_wrapper.get("data", {}) if data_wrapper else {}

        if data:
            # 개발사 / 퍼블리셔
            developer = ", ".join(data.get("developers", [])) if data.get("developers") else ""
            publisher = ", ".join(data.get("publishers", [])) if data.get("publishers") else ""

            # 무료 / 가격 / 상태 판정
            is_free = bool(data.get("is_free", False))
            if is_free:
                price = 0
                sale_status = "FREE"
            elif "price_overview" in data:
                try:
                    price = data["price_overview"]["final"] / 100.0
                    sale_status = "PAID"
                except Exception:
                    price = ""
                    sale_status = "UNKNOWN"
            else:
                # price_overview 없음 & is_free False -> 상점에서 제거되었거나 판매중 아님
                price = ""
                sale_status = "DELISTED"

            # 장르
            if "genres" in data and isinstance(data["genres"], list):
                genres = ", ".join([x.get("description","") for x in data["genres"]])

            # 출시일
            release_date = data.get("release_date", {}).get("date", "")

            # 한국어 지원 (문자열에 'Korean' 포함 여부 검사)
            if isinstance(data.get("supported_languages", ""), str) and "Korean" in data.get("supported_languages", ""):
                korean_support = "YES"

            # 메타크리틱 점수
            metacritic = data.get("metacritic", {}).get("score", "")

        # hours_per_price: price값이 숫자이고 PAID일 때만 계산
        if sale_status == "PAID" and isinstance(price, (int, float)) and price > 0:
            hours_per_price = round(total_hours / price, 3)
        else:
            hours_per_price = ""

        row = [
            appid,
            name,
            total_hours,
            recent_hours,
            recent_played,
            price,
            hours_per_price,
            is_free,
            sale_status,
            developer,
            publisher,
            genres,
            release_date,
            korean_support,
            metacritic,
            store_url
        ]

        ws.append(row)

        # 가운데 정렬 (해당 행)
        row_idx = ws.max_row
        for col in range(1, len(headers) + 1):
            ws.cell(row=row_idx, column=col).alignment = center_align

        print(f"[{idx}/{total}] 작성: {appid} - {name}")
        time.sleep(0.25)  # rate limit 완화

    # 필터 활성화 & 헤더 고정
    ws.auto_filter.ref = ws.dimensions
    ws.freeze_panes = "A2"

    # 열 너비 자동 조절
    for col in ws.columns:
        max_length = 0
        col_letter = get_column_letter(col[0].column)
        for cell in col:
            try:
                if cell.value is not None:
                    # 숫자는 적절한 폭, 문자열은 길이에 따라
                    val = str(cell.value)
                    # 한글/멀티바이트 간단 보정: len 대신 가중치 적용
                    length = 0
                    for ch in val:
                        # 한글/한자/일본어이면 2, 그 외 1
                        if ord(ch) > 255:
                            length += 2
                        else:
                            length += 1
                    max_length = max(max_length, length)
            except Exception:
                pass
        # 최소 너비 보장
        ws.column_dimensions[col_letter].width = (max_length + 2)

    # 저장
    try:
        wb.save(OUTPUT_XLSX)
        print("저장 완료:", OUTPUT_XLSX)
    except PermissionError:
        print("PermissionError: 파일을 열어두었거나 쓰기 권한이 없습니다. 엑셀에서 파일을 닫고 재시도하세요.")
    except Exception as e:
        print("파일 저장 중 오류:", e)

if __name__ == "__main__":
    main()
